## Preparation

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
#import packages
import os
import sqlite3
import pandas as pd

In [2]:
# Kemudian kita definisikan path folder dimana file database kita disimpan
path = 'D:\Kuliah\MBKM\Data Analyst - Narasio\Kodingan\Week 4\Dataset' #silahkan masukkan path sesuai dengan letak data di folder drive masing-masing

In [3]:
# untuk mengecek ada file apa saja didalam path yang digunakan
os.listdir(path)

['Breakoutroom Meet 9.ipynb',
 'Dataset - SQL for Data Analysis.zip',
 'Mainroom_Meet_9.ipynb',
 'rexon_metals.db',
 'transportation_sales.db',
 'weather_stations.db']

In [7]:
# Kemudian kembali membuat connection ke database rexon_metals yang udah ada
conn = sqlite3.connect(path + 'rexon_metals.db')

# Membuat cursor object untuk memanggil SQL statement
cur = conn.cursor()

cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

ValueError: parameters are of unsupported type

In [ ]:
# Kemudian kembali membuat connection ke database transportation_sales yang udah ada
conn_2 = sqlite3.connect(path + 'transportation_sales.db')

# Membuat cursor object untuk memanggil SQL statement
cur_2 = conn_2.cursor()

## Subquery, A Review

In [ ]:
# mengetahui total hasil penjualan setiap produk
pd.read_sql_query("""
                   SELECT 
                      p.PRODUCT_ID,
                      CASE WHEN p.PRICE*tq.TOTAL_QTY IS NULL THEN 0 ELSE p.PRICE*tq.TOTAL_QTY END AS SALES
                   FROM PRODUCT p
                   LEFT JOIN
                     (SELECT
                        PRODUCT_ID, 
                        SUM(ORDER_QTY) AS TOTAL_QTY
                      FROM CUSTOMER_ORDER
                      GROUP BY 1
                      ) tq
                    ON tq.PRODUCT_ID=p.PRODUCT_ID
                  """, conn)

,PRODUCT_ID,SALES
0,1,3379.50
1,2,2838.00
2,3,0.00
3,4,4616.25
4,5,1200.00
5,6,0.00
6,7,0.00
7,8,0.00
8,9,0.00


In [ ]:
# menambahkan tanggal pertama kali pengiriman pada masing-masing customer di tabel customer
pd.read_sql_query("""
                  SELECT 
                    c.*,
                    fod.FIRST_SHIP_DATE
                  FROM CUSTOMER c
                  LEFT JOIN  
                     (SELECT
                        CUSTOMER_ID, 
                        MIN(SHIP_DATE) AS FIRST_SHIP_DATE
                      FROM CUSTOMER_ORDER
                      GROUP BY 1
                      ) fod
                  ON fod.CUSTOMER_ID = c.CUSTOMER_ID
                  """, conn)

,CUSTOMER_ID,NAME,REGION,STREET_ADDRESS,CITY,STATE,ZIP,FIRST_SHIP_DATE
0,1,LITE Industrial,Southwest,729 Ravine Way,Irving,TX,75014,2015-05-18
1,2,Rex Tooling Inc,Southwest,6129 Collie Blvd,Dallas,TX,75201,None
2,3,Re-Barre Construction,Southwest,9043 Windy Dr,Irving,TX,75032,2015-05-20
3,4,Prairie Construction,Southwest,264 Long Rd,Moore,OK,62104,None
4,5,Marsh Lane Metal Works,Southeast,9143 Marsh Ln,Avondale,LA,79782,2015-05-22


# CTE

In [ ]:
# Kemudian kita dapat execute SQL statement. Kali ini kita akan mengecek terdapat tabel apa saja di rexon_metals.db
cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

[('PRODUCT',), ('sqlite_sequence',), ('CUSTOMER',), ('CUSTOMER_ORDER',), ('stocks',)]


In [ ]:
# Kemudian kita dapat execute SQL statement. Kali ini kita akan mengecek terdapat tabel apa saja di transportation_sales.db
cur_2.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur_2.fetchall())

[('Customers',), ('Products',), ('Orders',), ('mytable',)]


### Total hasil penjualan produk transportasi per bulan, per customer

In [ ]:
# Step 1: mengetahui total kuantiti terjual
pd.read_sql_query("""
                  WITH qty AS 
                      (SELECT
                        CUSTOMER_ID, STATUS,
                        SUM(SALES) AS TOTAL_SALES
                      FROM Orders
                      GROUP BY 1,2)
                  SELECT * FROM qty    
                  """, conn_2)

,CUSTOMER_ID,STATUS,TOTAL_SALES
0,1,Cancelled,3188.64
1,1,Shipped,10140.00
2,2,On Hold,6893.91
3,2,Shipped,10955.81
4,3,On Hold,2871.00
5,3,Shipped,2597.39
6,4,Cancelled,3884.34
7,4,Shipped,2434.56
8,5,On Hold,5404.62
9,5,Shipped,12037.50


In [ ]:
# Step 2: mengalikan kuantiti terjual dengan harga jual
pd.read_sql_query("""
                   WITH tq AS 
                      (SELECT
                        CUSTOMER_ID, STATUS,
                        SUM(SALES) AS TOTAL_SALES
                      FROM Orders
                      GROUP BY 1,2)

                      WITH tq AS 
                      (SELECT
                        CUSTOMER_ID, STATUS,
                        SUM(SALES) AS TOTAL_SALES
                      FROM Orders
                      GROUP BY 1,2)

                   SELECT 
                      p.CUSTOMERNAME,STATUS,
                      TOTAL_SALES
                   FROM Customers p
                   LEFT JOIN tq ON tq.CUSTOMER_ID=p.Customer_ID
                  """, conn_2)

,CUSTOMERNAME,STATUS,TOTAL_SALES
0,Land of Toys Inc.,Cancelled,3188.64
1,Land of Toys Inc.,Shipped,10140.00
2,Reims Collectables,On Hold,6893.91
3,Reims Collectables,Shipped,10955.81
4,Lyon Souveniers,On Hold,2871.00
5,Lyon Souveniers,Shipped,2597.39
6,Toys4GrownUps.com,Cancelled,3884.34
7,Toys4GrownUps.com,Shipped,2434.56
8,Corporate Gift Ideas Co.,On Hold,5404.62
9,Corporate Gift Ideas Co.,Shipped,12037.50
